In [1]:
import numpy as np
import os
os.chdir('/Users/magictavern/Documents/kaggkle/all')
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import accuracy_score
import lightgbm as lgb

/Users/magictavern/anaconda3/lib/python3.6/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.1) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
test = pd.read_csv('test.csv')
train= pd.read_csv('train.csv')
full=pd.concat([train,test],ignore_index=True)

/Users/magictavern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  This is separate from the ipykernel package so we can avoid doing imports until


In [3]:
full.Fare.fillna(full.Fare.value_counts().index[0],inplace=True)
full.Embarked.fillna(full.Embarked.value_counts().index[0],inplace=True)
full.Cabin.fillna('None',inplace=True)
full.Age.fillna(full.Age.median(),inplace=True)
full.Survived.fillna(-1,inplace=True)
sex_map={'male':0,'female':1}
full.Sex=full.Sex.map(sex_map)
def get_t(x):
    y=x.split(',')[1].split('.')[0]
    return y
full.Name=full.Name.map(get_t)
title_map=pd.Series(range(len(full.Name.value_counts())),index=full.Name.value_counts().index)
full.Name=full.Name.map(title_map)
full['Family']=full['SibSp']+full['Parch']
del full['Ticket']

In [4]:
cabin_map=pd.Series(range(len(full.Cabin.value_counts())),index=full.Cabin.value_counts().index)
full['Cabin']=full['Cabin'].map(cabin_map)
embarked_map=pd.Series(range(len(full.Embarked.value_counts())),index=full.Embarked.value_counts().index)
full['Embarked']=full['Embarked'].map(embarked_map)
cat_feature=['Cabin','Embarked','Name','Parch','Pclass','Sex','SibSp','Family']
result=full[full['Survived']==-1].pop('PassengerId')
del full['PassengerId']

In [5]:
train_data=full[full['Survived']!=-1]
test_data=full[full['Survived']==-1]
del test_data['Survived']
train_data['Survived']=train_data.Survived.astype(int)

/Users/magictavern/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [6]:
y=train_data.pop('Survived')
X_train,X_eval,y_train,y_eval=train_test_split(train_data,y,test_size=0.2,random_state=0)
lgb_train = lgb.Dataset(X_train, y_train,free_raw_data=False)
lgb_eval = lgb.Dataset(X_eval, y_eval,reference=lgb_train,free_raw_data=False)

In [7]:
params = {    'boosting_type': 'gbdt', 
    'objective': 'binary', 
              'learning_rate': 0.1,
          'is_unbalance': True 
         }
params['metrics']='binary_logloss'
best_score = float('Inf')
best_params = {}
cat_feature=['Cabin','Embarked','Name','Parch','Pclass','Sex','SibSp','Family']
print("调参1：num_leaves/max_bin")
for num_leaves in range(31,131,9):
    for max_bin in range(55,256,50):
        
        params['num_leaves'] = num_leaves
        params['max_bin'] = max_bin
            
        cv_results = lgb.cv(params, lgb_train,num_boost_round=1000, nfold=5,stratified=False,
        early_stopping_rounds=50, verbose_eval=False,categorical_feature=cat_feature )
            
        mean_binary_logloss = pd.Series(cv_results['binary_logloss-mean']).min()
            
        if mean_binary_logloss <= best_score:
            best_score = mean_binary_logloss
            best_params['num_leaves'] = num_leaves
            best_params['max_bin'] = max_bin
                
params['num_leaves'] = best_params['num_leaves']
params['max_bin'] = best_params['max_bin']

print('best params:', best_params)
print('best cv score:', best_score)

print("调参2：feature_fraction/min_data_in_leaf")
for feature_fraction in [0.6,0.8,1.0]:
    for min_data_in_leaf in range(20,101,10):
        
            params['feature_fraction'] =feature_fraction
            params['min_data_in_leaf'] = min_data_in_leaf
            
            cv_results = lgb.cv(params, lgb_train,num_boost_round=1000, nfold=5, stratified=False,
            early_stopping_rounds=50, verbose_eval=False,categorical_feature=cat_feature)
                    
            mean_binary_logloss = pd.Series(cv_results['binary_logloss-mean']).min()
            
            if mean_binary_logloss <= best_score:
                best_score = mean_binary_logloss
                best_params['feature_fraction']=feature_fraction
                best_params['min_data_in_leaf'] = min_data_in_leaf

params['min_data_in_leaf'] = best_params['min_data_in_leaf']
params['feature_fraction'] = best_params['feature_fraction']

print('best params:', best_params)
print('best cv score:', best_score)

print("调参3：bagging")
for bagging_fraction in [0.6,0.8,1.0]:
    for bagging_freq in range(0,11,1):
        params['bagging_fraction'] = bagging_fraction
        params['bagging_freq'] = bagging_freq
            
        cv_results = lgb.cv(params, lgb_train,num_boost_round=1000, nfold=5,stratified=False,
        early_stopping_rounds=50, verbose_eval=False,categorical_feature=cat_feature)
                    
        mean_binary_logloss = pd.Series(cv_results['binary_logloss-mean']).min()
            
        if mean_binary_logloss <= best_score:
            best_score = mean_binary_logloss
            best_params['bagging_fraction'] = bagging_fraction
            best_params['bagging_freq'] = bagging_freq

params['bagging_fraction'] = best_params['bagging_fraction']
params['bagging_freq'] = best_params['bagging_freq']

print('best params:', best_params)
print('best cv score:', best_score)

print("调参4：l1/l2")
for lambda_l1 in [0.0,0.01,0.1,1.0,10,100,1000]:
    for lambda_l2 in [0.0,0.01,0.1,1.0,10,100,1000]:
        
        params['lambda_l1'] = lambda_l1
        params['lambda_l2'] = lambda_l2
            
        cv_results = lgb.cv(params, lgb_train,num_boost_round=1000, nfold=5, stratified=False,
        early_stopping_rounds=50, verbose_eval=False,categorical_feature=cat_feature)
                    
        mean_binary_logloss = pd.Series(cv_results['binary_logloss-mean']).min()
            
        if mean_binary_logloss <= best_score:
            best_score = mean_binary_logloss
            best_params['lambda_l1'] = lambda_l1
            best_params['lambda_l2'] = lambda_l2

params['lambda_l1'] = best_params['lambda_l1']
params['lambda_l2'] = best_params['lambda_l2']

print('best params:', best_params)
print('best cv score:', best_score)

调参1：num_leaves/max_bin


/Users/magictavern/anaconda3/lib/python3.6/site-packages/lightgbm/basic.py:1194: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['Cabin', 'Embarked', 'Family', 'Name', 'Parch', 'Pclass', 'Sex', 'SibSp']
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))


best params: {'num_leaves': 130, 'max_bin': 255}
best cv score: 0.4392241684202651
调参2：feature_fraction/min_data_in_leaf
best params: {'num_leaves': 130, 'max_bin': 255, 'feature_fraction': 1.0, 'min_data_in_leaf': 30}
best cv score: 0.43517037970451905
调参3：bagging
best params: {'num_leaves': 130, 'max_bin': 255, 'feature_fraction': 1.0, 'min_data_in_leaf': 30, 'bagging_fraction': 1.0, 'bagging_freq': 6}
best cv score: 0.43517037970451894
调参4：l1/l2
best params: {'num_leaves': 130, 'max_bin': 255, 'feature_fraction': 1.0, 'min_data_in_leaf': 30, 'bagging_fraction': 1.0, 'bagging_freq': 6, 'lambda_l1': 1.0, 'lambda_l2': 0.01}
best cv score: 0.43462257839522334


In [13]:
params['learning_rate']=0.01
lgbm=lgb.train(
          params,                     # 参数字典
          lgb_train,                  # 训练集
          valid_sets=[lgb_train,lgb_eval],
    valid_names=['train','evals'],
          num_boost_round=2000,       # 迭代次数
          early_stopping_rounds=50,# 早停次数
         verbose_eval=False,categorical_feature=cat_feature
          )
feat_importance=pd.Series(lgbm.feature_importance(),index=test_data.columns)
y_pred=lgbm.predict(test_data) # 输出概率
pre_y=pd.Series(y_pred,index=result.values)
result=pre_y.map(lambda x: 1 if x >0.5 else 0).reset_index()
result.rename(columns={'index':'PassengerId',0:'Survived'},inplace=True)
result.to_csv('titanic_pred20.csv',index=False)